In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


### 데이터 전처리

In [2]:
data = pd.read_csv('../ssd_data/imputated_data/imputated_data_MB1.csv')

data

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
0,2019-01-01,31346.0,100.0,0.0,97.0,12769.0,99.0,13.0,99.0,31.0,...,28.0,200.0,0.0,100.0,0.0,99.0,3.003245e+10,99.0,2.761588e+10,0.0
1,2019-01-02,31346.0,100.0,0.0,97.0,12792.0,99.0,13.0,99.0,31.0,...,28.0,200.0,0.0,100.0,0.0,99.0,3.058151e+10,99.0,2.778027e+10,0.0
2,2019-01-03,31346.0,100.0,0.0,97.0,12816.0,99.0,13.0,99.0,31.0,...,28.0,200.0,0.0,100.0,0.0,99.0,3.106840e+10,99.0,2.795260e+10,0.0
3,2019-01-04,31346.0,100.0,0.0,97.0,12840.0,99.0,13.0,99.0,32.0,...,28.0,200.0,0.0,100.0,0.0,99.0,3.144897e+10,99.0,2.812494e+10,0.0
4,2019-01-05,31346.0,100.0,0.0,97.0,12864.0,99.0,13.0,99.0,32.0,...,28.0,200.0,0.0,100.0,0.0,99.0,3.179422e+10,99.0,2.829816e+10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50313,2019-06-26,8599.0,100.0,0.0,96.0,17168.0,99.0,21.0,91.0,623.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201000e+12,99.0,8.356453e+11,0.0
50314,2019-06-27,8599.0,100.0,0.0,96.0,17192.0,99.0,21.0,91.0,623.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201312e+12,99.0,8.362658e+11,0.0
50315,2019-06-28,8599.0,100.0,0.0,96.0,17216.0,99.0,21.0,91.0,624.0,...,31.0,200.0,0.0,100.0,0.0,99.0,2.201585e+12,99.0,8.371074e+11,0.0
50316,2019-06-29,8599.0,99.0,2.0,96.0,17240.0,99.0,21.0,91.0,624.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201871e+12,99.0,8.379656e+11,0.0


In [14]:
#label 

[32240.0,
 8656.0,
 38576.0,
 43191.0,
 28588.0,
 3831.0,
 4040.0,
 30277.0,
 39983.0,
 33543.0,
 30640.0,
 35780.0,
 1850.0,
 41400.0,
 36235.0,
 36231.0,
 36228.0,
 36225.0,
 36227.0,
 24395.0,
 30644.0,
 31304.0,
 36232.0,
 30646.0,
 31804.0,
 20920.0,
 19420.0,
 5351.0,
 29917.0,
 39939.0,
 25624.0,
 37709.0,
 13686.0,
 25063.0,
 5434.0,
 35737.0,
 34853.0,
 14877.0,
 31295.0,
 37479.0,
 39666.0,
 44338.0,
 25763.0,
 29290.0,
 42313.0,
 17819.0,
 17817.0,
 3688.0,
 38451.0,
 42119.0,
 22437.0,
 3686.0,
 42311.0,
 44335.0,
 23050.0,
 23056.0,
 5876.0,
 12786.0,
 22440.0,
 37471.0,
 22439.0,
 42323.0,
 28302.0,
 39655.0,
 38452.0,
 42935.0,
 29280.0,
 3693.0,
 25764.0,
 37476.0,
 24759.0,
 3692.0,
 5883.0,
 8789.0,
 42112.0,
 42319.0,
 8149.0,
 34294.0,
 42309.0,
 12781.0,
 8061.0,
 9758.0,
 15078.0,
 7981.0,
 25759.0,
 5581.0,
 7045.0,
 36353.0,
 3690.0,
 33733.0,
 22299.0,
 8791.0,
 28296.0,
 37740.0,
 7112.0,
 34251.0,
 16306.0,
 13564.0,
 22229.0,
 531.0,
 15274.0,
 29467.0,
 150

In [15]:
import numpy as np
def create_sequences(data, window_size):
    X = []
    y = []

    failed_disk_id = list(data[data['label']==1]['disk_id'].values)

    # 각 disk_id에 대해 처리
    for disk_id in data['disk_id'].unique():
        disk_data = data[data['disk_id'] == disk_id].sort_values('index')  # ds로 정렬
        disk_values = disk_data.drop(columns=['disk_id', 'index']).values  # 'disk_id'와 'ds' 컬럼을 제외하고 numpy array로 변환

        if disk_id in failed_disk_id:
            failed_index = disk_val
            X.append(disk_values[-window_size:,:-1])
            y.append(disk_values[,-1])
        
        # 이 disk_id에 대해 시퀀스 생성
        for i in range(len(disk_values) - window_size):
            X.append(disk_values[i:i + window_size, :-1])  # 마지막 차원은 특성(feature)을 의미합니다. 마지막 열('fail')을 제외합니다.
            y.append(disk_values[i + window_size, -1])  # 예를 들어 'fail' 상태가 마지막 특성일 경우입니다.

    return np.array(X), np.array(y)

In [16]:
X,y=create_sequences(data[data['disk_id']==36235.0],30)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (152, 30) + inhomogeneous part.

## Conv 1D + LSTM

- Conv 1D 를 통해, 30일의 feature extraction(FE)
- FE된 feature map들의 시계열성 특징을 LSTM을 통해 추출
- 마지막 Fully Connected layer를 통해, Classification 진행(healthy / fail)

In [ ]:
model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None,30, 28)))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(50, activation='relu'))
model_cnn_lstm.add(Dense(1, activation='sigmoid'))
model_cnn_lstm.compile(loss='binary_crossentropy', optimizer='adam')

In [20]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout

# TensorFlow eager execution 설정
tf.config.run_functions_eagerly(True)

# 예시로서 X_train 데이터 생성
# X_train 데이터의 형태: (batch_size, 6, 30, 28)
batch_size = 100
seq_length = 6
timesteps = 30
num_features = 28

# 임의의 데이터로 예시를 보여주기 위해 랜덤한 값을 사용
X_train = np.random.randn(batch_size, seq_length, timesteps, num_features)
y_train = np.random.randint(0, 2, size=(batch_size, 1))

# 모델 정의
model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu'), input_shape=(None,30, 28)))
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
# model_cnn_lstm.add(TimeDistributed(Dropout(0.5)))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(64, activation='relu'))
model_cnn_lstm.add(Dense(64, activation='relu'))
# model_cnn_lstm.add(Dropout(0.5))
model_cnn_lstm.add(Dense(1, activation='sigmoid'))
# 모델 컴파일
model_cnn_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# TensorFlow eager execution 재설정
tf.config.run_functions_eagerly(False)

# 학습S
history = model_cnn_lstm.fit(X_train, y_train, epochs=5, verbose=0, batch_size=32)

In [21]:
model_cnn_lstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_4 (TimeDis  (None, None, 28, 32)     2720      
 tributed)                                                       
                                                                 
 time_distributed_5 (TimeDis  (None, None, 26, 64)     6208      
 tributed)                                                       
                                                                 
 time_distributed_6 (TimeDis  (None, None, 13, 64)     0         
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, None, 832)        0         
 tributed)                                                       
                                                                 
 lstm_1 (LSTM)               (None, 64)               

In [ ]:
# 임의의 데이터로 예시를 보여주기 위해 랜덤한 값을 사용
X_test = np.random.randn(batch_size, seq_length, timesteps, num_features)
y_test = np.random.randint(0, 2, size=(batch_size, 1))

## 데이터 전처리

In [ ]:
predicted=model_cnn_lstm.predict(X_test)

4/4 [==============================] - 0s 8ms/step


In [ ]:
y_hat = np.array([])  # 빈 배열로 초기화

for i in range(len(X_test)):
    # 예측 결과를 기반으로 y_hat에 원소를 추가
    if model_cnn_lstm.predict(X_test[i:i+1]) > 0.5:
        y_hat = np.concatenate((y_hat, np.array([1])), axis=0)
    else:
        y_hat = np.concatenate((y_hat, np.array([0])), axis=0)

1/1 [==============================] - 0s 23ms/step


In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_hat)

0.5